# Testing merges 

In [1]:
import numpy as np
import pandas as pd
import math
from siuba import *
from calitp import *
import intake
import data_prep

import shared_utils
from plotnine import *
import altair as alt
import altair_saver
from shared_utils import altair_utils 

pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = 250

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_cleaned_vehiclesdata()
organizations = data_prep.load_cleaned_organizations_data()

/home/jovyan/data-analyses/5311/data_prep.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

# Merge NTD with GTFS  --> vehicle_gtfs
<b> NOTE</b>:
I had to manually add NTD ID to the following agencies in the original CSV file called "cleaned organizations.csv"

* Butte County Association of Governments	90208
* City of Dixon	9402-91041
* City of Fairfield	90092
* City of Ridgecrest	9R02-91006
* City of Wasco	9R02-99426
* Glenn County	9R02-91088
* Mariposa County	9R02-91082
* Modoc Transportation Authority 	9R02-91008
* Palo Verde Valley Transit Agency	9R02-99454
* San Benito County Local Transportation Authority	9R02-91009
* San Joaquin Regional Transit District	90012
* Tuolumne County Transit Agency	9402-035


In [3]:
#trying the iloc way 
#organizations.loc[(organizations['name'] == 'Butte County Association of Governments'), "ntd_id"] = "90208"
#vehicles.loc[(vehicles['agency'] == 'Butte County Association of Governments'), "ntd_id"] = "90208"

#organizations.loc[(organizations['name'] == 'City of Fairfield'), "ntd_id"] = "90092"
#vehicles.loc[(vehicles['agency'] == 'City of Fairfield, California, dba: Fairfield and Suisun Transit'), "ntd_id"] = "90092"

In [4]:
#merging the 2 datasets together
vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'], indicator=True)

In [5]:
vehicle_agencies = vehicles_gtfs['agency'].drop_duplicates().tolist()

In [6]:
vehicles_gtfs['_merge'].value_counts()

both          192
left_only      26
right_only      0
Name: _merge, dtype: int64

In [7]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

# Merging -> Vehicle & GTFS with Black Cat.

In [8]:
#left merge, Black Cat on the left
Test1 = pd.merge(df_5311, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['name'], indicator=True)

In [9]:
len(Test1)

798

In [10]:
# Counting left only and both
Test1['_merge'].value_counts()

both          495
left_only     303
right_only      0
Name: _merge, dtype: int64

In [11]:
#finding the agencies that are in left only.
Left_only = Test1[(Test1._merge.str.contains("left_only", case= False))] 
Left_orgs = Left_only['organization_name'].drop_duplicates().tolist()

In [12]:
# Filter out left only matches
m2 = Test1[~Test1.organization_name.isin(Left_orgs)]

In [13]:
#check that filter worked ok
m2.shape

(495, 45)

In [14]:
m2.sample(3)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,_merge
475,2019,Section 5311,CA-2020-015 | 0020000057,2019,Humboldt Transit Authority,BCG0001488,Purchase Replacement Std 35 Ft Bus,111202,64BC19-00926,225207.00,225207.00,0.00,225207.00,0,Open,Humboldt Transit Authority,9R02-91036,Rural Reporter,28.00,4.00,4.00,1.00,0.00,0.00,0.00,0.00,37.00,6.72,0.00,NaN,0.00,16.00,0.00,8.00,0.00,13.00,0.00,32.00,0.00,13.00,45.00,Humboldt Transit Authority,135.00,"needed,ok,ok,ok",both
327,2018,Section 5311,CA-2018-114 | 0019000012,2018,City of Woodlake,BCG0001286,Operating Assistance Sliding Scale,300902,64BO19-00853,2709.00,2709.00,2709.00,0.00,0,Open,City of Woodlake,9R02-91079,Rural Reporter,1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,3.00,13.33,0.00,NaN,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,0.00,3.00,3.00,City of Woodlake,371.00,needed,both
261,2017,Section 5311(f),CA-2017-168 | 0017000259-F,2017,Monterey-Salinas Transit,BCG0000209,Operating Assistance (Line 85),300902,64CO17-00453,165989.00,255099.00,165989.00,0.00,0,Open,Monterey-Salinas Transit,90062,Full Reporter,151.00,15.00,14.00,16.00,0.00,0.00,0.00,0.00,196.00,7.13,317142.10,NaN,0.00,93.00,0.00,38.00,0.00,65.00,0.00,186.00,0.00,65.00,251.00,Monterey-Salinas Transit,208.00,"needed,needed,needed,needed,research,needed",both


### Failed Matches, crosswalk manually

In [15]:
#making a data frame with only failed merges out out of original Black Cat
fail = df_5311[df_5311.organization_name.isin(Left_orgs)]

In [16]:
fail.shape

(303, 15)

In [17]:
#crosswalk dictionary for replacing organization name
crosswalk = {'City of Chowchilla ': 'City of Chowchilla, dba: Chowchilla Area Transit ',
 'City of Dinuba ':  'City of Dinuba',
 'Modoc Transportation Agency': 'Modoc Transportation Agency',
 'Butte County Association of Governments/ Butte Regional Transit': 'Butte County Association of Governments',
 'Calaveras County Public Works':  'Calaveras Transit Agency',
 'City of Escalon ':  'City of Escalon, dba: eTrans',
 'County of Mariposa':  'Mariposa County Transit, dba: Mari-Go',
 'County of Shasta Department of Public Works':  'County of Shasta Department of Public Works',
 'County of Siskiyou': 'County of Siskiyou, dba: Siskiyou County Transit',
 'County of Tulare': 'Tulare County Area Transit',
 'Eureka Transit Service':  'City of Eureka, dba: Eureka Transit Service',
 'Kern Regional Transit':  'Kern Regional Transit',
 'Livermore Amador Valley Transit Authority':  'Livermore / Amador Valley Transit Authority',
 'Placer County Public Works (TART & PCT)': 'County of Placer, dba: Placer County Department of Public Works',
 'Plumas County Transportation Commission': 'Plumas County Transportation Commission',
 'San Luis Obispo Regional Transit Authority':  'San Luis Obispo Regional Transit Authority',
 'Sonoma County Transit':  'County of Sonoma, dba: Sonoma County Transit',
 'Sunline Transit Agency':  'SunLine Transit Agency',
 'Tehama County Transit Agency': 'Tehama County',
 'Trinity County Department of Transportation ':  'Trinity County',
 'Tuolumne County Transit Agency (TCTA)':  'Tuolumne County Transit',
 'Amador Transit':  'Amador Regional Transit System',
 'City of Corcoran - Corcoran Area Transit':  'City of Corcoran, dba: Corcoran Area Transit',
 'Yosemite Area Regional Transportation System ':  'Yosemite Area Regional Transportation System',
 'County Connection (Central Contra Costa Transit Authority)': 'Central Contra Costa Transit Authority, dba: COUNTY CONNECTION',
 'Calaveras Transit Agency ': 'Calaveras Transit Agency'}

In [18]:
#failed merges
fail['organization_name'].replace(crosswalk, inplace= True)

/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
#Merging the failed organizations to vehicles 
Test2 = pd.merge(fail, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['agency'], indicator=True)

In [20]:
Test2['_merge'].value_counts()

both          248
left_only      55
right_only      0
Name: _merge, dtype: int64

In [21]:
#finding the agencies that are in left only...make sure these are the ones we aren't sure have any matches.
Left_only = Test2[(Test2._merge.str.contains("left_only", case= False))]
#find failed agencies 

Left_only_orgs= Left_only['organization_name'].drop_duplicates().tolist()
Left_only_orgs

['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Klamath Trinity Non-Emergency Transportation\u200b',
 'Greyhound Lines, Inc.']

In [22]:
#deleting Klamath off since it has GTFS, we only want agencies that do not appear in both GTFS & NTD 
L_only_2_orgs = ['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Greyhound Lines, Inc.']

In [23]:
#appending failed matches to the first data frame
BC_GTFS_NTD = m2.append(Test2, ignore_index=True)

In [24]:
#checking that rows match up
BC_GTFS_NTD.shape

(798, 45)

In [25]:
#checking that organizations are here
BC_GTFS_NTD['organization_name'].nunique()

87

In [26]:
#drop  merge
BC_GTFS_NTD = BC_GTFS_NTD.drop(columns=['_merge'])

### Flag agencies that appear in Black Cat Only

In [27]:
def BC_only(row):
    if row.organization_name in Left_only_orgs:
        return '1'
    else: 
        return '0'  

In [28]:
BC_GTFS_NTD["Is_Agency_In_BC_Only_1_means_Yes"] = BC_GTFS_NTD.apply(lambda x: BC_only(x), axis=1)

In [29]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("greyhound", case= False))].head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes
578,2017,Section 5311(f),CA-2017-168 | 0017000259-F,2017,"Greyhound Lines, Inc.",BCG0000173,Operating Assistance - Route 1 Service from Ar...,300902,64CO17-00447,161293.00,247887.00,161293.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


### Replacing Klamath
* Klamath has GTFS but no NTD records

In [30]:
BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"

In [31]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("Klamath", case= False))].head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes
544,2017,5311(f) Cont,TBD,2017,Klamath Trinity Non-Emergency Transportation​,BCG0000168,Operating Assistance,300902,64CO17-00467,8344.00,8344.00,8344.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,436,needed,1


### Getting ok/needed/research out of original GTFS data frame

In [32]:
#look at original GTFS status for all the different varities...
BC_GTFS_NTD['gtfs_schedule_status'].unique()

array(['ok,needed,needed',
       'needed,needed,ok,ok,needed,ok,ok,needed,needed,needed,needed,needed,ok,ok,ok,needed,ok,needed,needed,needed,needed,needed',
       'ok', 'needed,ok,needed,ok', 'needed,ok,ok,ok',
       'need long-term solution,needed', 'ok,needed', 'needed',
       'ok,needed,ok', 'needed,ok', 'need long-term solution', nan,
       'ok,needed,ok,ok,ok,needed,needed,needed,needed',
       'ok,needed,needed,needed,needed', 'needed,needed,needed,needed',
       'needed,ok,needed,needed,needed,needed,needed,needed,needed',
       'ok,needed,needed,needed,needed,needed',
       'needed,needed,needed,needed,research,needed', 'ok,research',
       'ok,needed,needed,ok,needed,needed,research,needed',
       'ok,ok,ok,needed,needed', 'needed,ok,needed',
       'ok,ok,ok,ok,needed,needed',
       'ok,needed,needed,needed,research,needed',
       'ok,ok,ok,ok,ok,ok,needed,needed', 'needed,needed,ok,ok',
       'needed,needed,needed', 'needed,ok,needed,needed', 'ok,ok',
       '

In [33]:
#create a new column
temp = BC_GTFS_NTD.gtfs_schedule_status.fillna("None")
BC_GTFS_NTD['GTFS_schedule_status_use'] = np.where(temp.str.contains("None"),"None",
                   np.where(temp.str.contains("ok"), "Ok",
                   np.where(temp.str.contains("long"), "Long-term solution needed",
                   np.where(temp.str.contains("research"), "Research", "Needed"))))


### Rating Fleet Size
* Using the aggregated df below, I did aggregate1['total_vehicles'].describe() and looked at the different percentiles to choose small, med, large, and no info

In [34]:
#Function
def fleet_size (row):
            if ((row.total_vehicles > 0) and (row.total_vehicles < 26)):
                return "Small"
            elif ((row.total_vehicles > 25) and (row.total_vehicles < 82)):
                return "Medium"
            elif ((row.total_vehicles > 82) and (row.total_vehicles < 1200)):
                return "Large"
            else:
                return "No Info"

In [35]:
BC_GTFS_NTD["fleet_size"] = BC_GTFS_NTD.apply(lambda x: fleet_size(x), axis=1)

In [36]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.fleet_size.str.contains("Info", case= False))]

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes,GTFS_schedule_status_use,fleet_size
505,2016,Section 5311,CA-2017-025 | 0017000158,2016,County of Los Angeles - Department of Public W...,BCG0000071,Operating Assistance,300902,64BO17-00390,400384.00,400384.00,400384.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None,No Info
507,2016,Section 5311,CA-2017-025 | 0017000158,2016,"County of Nevada Public Works, Transit Service...",BCG0000272,Operating Assistance,300902,64BO17-00392,440467.00,440467.00,440467.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None,No Info
508,2016,Section 5311,CA-2017-025 | 0017000158,2016,County of Sacramento Department of Transportation,BCG0000171,Operating Assistance for Sacramento County rur...,300902,64BO17-00393,275122.00,275122.00,275122.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None,No Info
513,2016,Section 5311,CA-2017-025 | 0017000158,2016,Glenn County Transportation Commission,BCG0000230,Operating Assistance,300902,64BO17-00400,150218.00,150218.00,150218.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None,No Info
529,2016,Section 5311,CA-2017-025 | 0017000158,2016,Stanislaus County Public Works - Transit Division,BCG0000085,Operating Assistance,300902,64BO17-00432,433556.00,433556.00,433556.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None,No Info
535,2016,Section 5311,CA-2017-025 | 0017000158,2017,Alpine County Community Development,BCG0000145,Alpine County Operating Assistance,300902,64BO17-00359,48000.00,48000.00,34673.12,13326.88,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None,No Info
542,2017,5311(f) Cont,TBD,2017,Fresno Council of Governments,BCG0000114,Operating Assistance,300902,64CO17-00464,300000.00,300000.00,237072.33,62927.67,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None,No Info
544,2017,5311(f) Cont,TBD,2017,Klamath Trinity Non-Emergency Transportation​,BCG0000168,Operating Assistance,300902,64CO17-00467,8344.00,8344.00,8344.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,436,needed,1,Needed,No Info
550,2017,Section 5311,CA-2017-168 | 0017000259,2017,Alpine County Community Development,BCG0001178,Operating Assistance Sliding Scale,300902,64BO17-00570,48000.00,48000.00,36382.04,11617.96,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None,No Info
557,2017,Section 5311,CA-2017-168 | 0017000259,2017,County of Los Angeles - Department of Public W...,BCG0001045,Operating Assistance Sliding Scale,300902,64BO17-00557,404396.00,404396.00,404396.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None,No Info


### Final Cleaning

In [37]:
#delete old columns
BC_GTFS_NTD2 = BC_GTFS_NTD.drop(columns=['gtfs_schedule_status','name','agency'])

#rename
BC_GTFS_NTD2 = BC_GTFS_NTD2.rename(columns = {'GTFS_schedule_status_use':'GTFS'})

In [38]:
#get agencies without any data to show up 
show_up = ['reporter_type']
for i in show_up:
    BC_GTFS_NTD2[i] = BC_GTFS_NTD2[i].fillna('None')

In [39]:
#change itp id to be float.
BC_GTFS_NTD2['itp_id'] = BC_GTFS_NTD2['itp_id'].fillna(0)
BC_GTFS_NTD2.loc[(BC_GTFS_NTD['itp_id'] == '436'), "itp_id"] = 436

### Parquet

In [40]:
#Parquet
BC_GTFS_NTD2.to_parquet("BC_GTFS_NTD.parquet")
#Adding to GCS (just in case)
#BC_GTFS_NTD2.to_parquet("gs://calitp-analytics-data/data-analyses/5311 /BC_GTFS_NTD.parquet")
#BC_GTFS_NTD2.to_csv("gs://calitp-analytics-data/data-analyses/5311 /BC_GTFS_NTD.csv")

# Taking a quick look


(2) % and # vehicles older vehicles (10, 15 yrs), avg age by fleet size buckets

(3) # doors (which means # validators) for vehicles over 10 yrs by GTFS status and by fleet size

In [41]:
BC_GTFS_NTD2.columns

Index(['grant_fiscal_year', 'funding_program', 'grant_number', 'project_year',
       'organization_name', 'upin', 'description', 'ali', 'contract_number',
       'allocationamount', 'encumbered_amount', 'expendedamount',
       'activebalance', 'closedoutbalance', 'project_status', 'ntd_id',
       'reporter_type', '_0_9', '_10_12', '_13_15', '_16_20', '_21_25',
       '_26_30', '_31_60', '_60plus', 'total_vehicles',
       'average_age_of_fleet__in_years_', 'average_lifetime_miles_per_vehicle',
       'sum_15plus', 'Automobiles', 'Bus', 'Other', 'Service', 'Train', 'Van',
       'automobiles_door', 'bus_doors', 'train_doors', 'van_doors',
       'doors_sum', 'itp_id', 'Is_Agency_In_BC_Only_1_means_Yes', 'GTFS',
       'fleet_size'],
      dtype='object')

In [42]:
#First grabbing only one row for each agency.
Aggregate1 = BC_GTFS_NTD2.groupby(['organization_name','reporter_type','GTFS', 'fleet_size', 'Is_Agency_In_BC_Only_1_means_Yes']).agg({'total_vehicles':'max',
       'average_age_of_fleet__in_years_':'max','doors_sum':'max','_0_9':'max', '_10_12':'max', '_13_15':'max', '_16_20':'max', '_21_25':'max',
       '_26_30':'max', '_31_60':'max', '_60plus':'max',
})

In [43]:
#reset index
Aggregate1 = Aggregate1.reset_index()

### (1) % and # vehicles over 10 yrs by GTFS status bucket. Another version of over 15 yrs

In [44]:
#duplicate df into vehicles
vehicles_gtfs_ten = Aggregate1.copy()

In [45]:
#Adding up by age group
vehicles_gtfs_ten['vehicles_older_than_9']= vehicles_gtfs_ten['_10_12'] + vehicles_gtfs_ten['_13_15'] + vehicles_gtfs_ten['_16_20'] + vehicles_gtfs_ten['_21_25'] + vehicles_gtfs_ten['_26_30'] + vehicles_gtfs_ten['_31_60'] + vehicles_gtfs_ten['_60plus']
vehicles_gtfs_ten['vehicles_older_than_16']= vehicles_gtfs_ten['_16_20'] + vehicles_gtfs_ten['_21_25'] + vehicles_gtfs_ten['_26_30'] + vehicles_gtfs_ten['_31_60'] + vehicles_gtfs_ten['_60plus']

#dividing by age group
vehicles_gtfs_ten['vehicles_percent_older_than_9'] = (vehicles_gtfs_ten['vehicles_older_than_9']/vehicles_gtfs_ten['total_vehicles'])*100
vehicles_gtfs_ten['vehicles_percent_older_than_16'] = (vehicles_gtfs_ten['vehicles_older_than_16']/vehicles_gtfs_ten['total_vehicles'])*100
vehicles_gtfs_ten['vehicles_percent_under_10'] = (vehicles_gtfs_ten['_0_9']/vehicles_gtfs_ten['total_vehicles'])*100

In [46]:
vehicles_gtfs_ten.columns

Index(['organization_name', 'reporter_type', 'GTFS', 'fleet_size',
       'Is_Agency_In_BC_Only_1_means_Yes', 'total_vehicles',
       'average_age_of_fleet__in_years_', 'doors_sum', '_0_9', '_10_12',
       '_13_15', '_16_20', '_21_25', '_26_30', '_31_60', '_60plus',
       'vehicles_older_than_9', 'vehicles_older_than_16',
       'vehicles_percent_older_than_9', 'vehicles_percent_older_than_16',
       'vehicles_percent_under_10'],
      dtype='object')

In [48]:
#ONLY for vehicles over 9 
GTFS_ten = vehicles_gtfs_ten.groupby(['GTFS']).agg({'vehicles_older_than_9':'sum','vehicles_percent_older_than_9':'median' })
GTFS_ten = GTFS_ten.reset_index()
GTFS_ten

,GTFS,vehicles_older_than_9,vehicles_percent_older_than_9
0,Long-term solution needed,7.00,27.27
1,Needed,34.00,25.00
2,None,51.00,0.00
3,Ok,1901.00,20.29
4,Research,45.00,22.96


In [50]:
data_prep.basic_bar_chart(GTFS_ten,'GTFS','vehicles_percent_older_than_9') 

alt.Chart(...)

### (2) % and # vehicles older vehicles (10, 15 yrs), avg age by fleet size buckets

In [51]:
#fleet size 9 years & older
fleet_ten = vehicles_gtfs_ten.groupby(['fleet_size']).agg({'vehicles_older_than_9':'sum','vehicles_percent_older_than_9':'median'})
fleet_ten = fleet_ten.reset_index()
fleet_ten

,fleet_size,vehicles_older_than_9,vehicles_percent_older_than_9
0,Large,1752.00,29.78
1,Medium,203.00,18.18
2,No Info,0.00,NaN
3,Small,83.00,16.23


### Looking at organizations & GTFS

In [52]:
Looking_at_GTFS = Aggregate1.groupby(['GTFS']).agg({'organization_name':'nunique'})
Looking_at_GTFS = Looking_at_GTFS.reset_index()
Looking_at_GTFS = Looking_at_GTFS.rename(columns = {'organization_name':'Count of Agencies by GTFS'})
Looking_at_GTFS

,GTFS,Count of Agencies by GTFS
0,Long-term solution needed,3
1,Needed,15
2,None,14
3,Ok,54
4,Research,1


In [53]:
data_prep.basic_bar_chart(Looking_at_GTFS,'GTFS','Count of Agencies by GTFS') 

alt.Chart(...)